In [3]:
import os
from os.path import join as opj

import numpy as np
import nibabel as nib
import glob
from nilearn.image import mean_img

In [8]:
from nilearn.masking import intersect_masks
from nipype.interfaces.afni import Calc

In [5]:
sub = 'sub-MONSTERA01'
n_trunc = 6
derivative_dir = '/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/'
fmriprep_base_dir = opj(derivative_dir, 'fmriprep/')
bold_dir=opj(fmriprep_base_dir, '%s/func/' % (sub))

In [6]:
session_list = []
for i in range(1,11):
    session_list.append('task-{:02d}'.format(i))

In [7]:
out_dir = opj(derivative_dir, 'preprocess/%s' % (sub))
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

## Mask intersection

In [6]:
mask_imgs = [x for x in glob.glob(os.path.join(fmriprep_base_dir, sub, 'func', sub+'_task-*_space-T1w_desc-brain_mask.nii.gz'))] 
mask_imgs.sort()
mask_imgs
overall_mask = intersect_masks(mask_imgs, threshold=1, connected=True)
mask_output = opj(out_dir, '%s_space-T1w_desc-brain_intersect_mask.nii.gz' % (sub))
overall_mask.to_filename(mask_output)


## Trim images

In [ ]:
for run in session_list:
    out_file=opj(out_dir, '%s_%s_space-T1w_desc-preproc_bold_trim%dTRs_centered-masked.nii.gz' % (sub, run, n_trunc))
    if os.path.exists(out_file): 
        continue
    else:
        # T1 space
        func_file = opj(bold_dir, "%s_%s_space-T1w_desc-preproc_bold.nii.gz"% (sub, run))
        
        epi_data=nib.load(func_file)
        epi=epi_data.get_fdata()
        #truncate
        epi_trunc =np.zeros((epi_data.shape[0], epi_data.shape[1], epi_data.shape[2], epi_data.shape[3]-n_trunc))
        epi_trunc[:, :, :, :] = epi[:,:,:,n_trunc:]

        print(epi_data.shape, '  ', epi_trunc.shape)
        
        dimsize=epi_data.header.get_zooms()
        print(dimsize)
        orig_dimsize=dimsize

        affine_mat = epi_data.affine  # What is the orientation of the data
        print(affine_mat)

        # Save the volume
        trim_output = opj(out_dir, '%s_%s_space-T1w_desc-preproc_bold_trim%dTRs.nii.gz' % (sub, run, n_trunc))
        
        bold_nii = nib.Nifti1Image(epi_trunc, affine_mat)
        hdr = bold_nii.header  # get a handle for the .nii file's header
        hdr.set_zooms((dimsize[0], dimsize[1], dimsize[2], dimsize[3]))
        nib.save(bold_nii, trim_output)
        
        # Center values
        ## Calculate mean images
        mean_img = mean_img(trim_output, target_affine=None, target_shape=None, verbose=0, n_jobs=1)
        mean_output = opj(out_dir, '%s_%s_space-T1w_desc-preproc_bold_trim%dTRs_mean-img.nii.gz' % (sub, run, n_trunc))
        mean_img.to_filename(mean_output)
        
        ## Using AFNI to center and set upper and lower bound
        calc = Calc()
        calc.inputs.in_file_a = trim_output
        calc.inputs.in_file_b = mean_output
        calc.inputs.in_file_c = mask_output
        
        calc.inputs.expr= 'c * min(200, a/b*100)*step(a)*step(b)'
        calc.inputs.out_file = out_file

        calc.cmdline 
        break

(96, 99, 84, 458)    (96, 99, 84, 452)
(1.702, 1.702, 1.7, 1.0)
[[  1.70200002   0.           0.         -81.10224915]
 [  0.           1.70200002   0.         -75.16112518]
 [  0.           0.           1.70000005 -58.7443161 ]
 [  0.           0.           0.           1.        ]]


In [14]:
run = session_list[0]
trim_output = opj(out_dir, '%s_%s_space-T1w_desc-preproc_bold_trim%dTRs.nii.gz' % (sub, run, n_trunc))
mean_output = opj(out_dir, '%s_%s_space-T1w_desc-preproc_bold_trim%dTRs_mean-img.nii.gz' % (sub, run, n_trunc))
mask_output = opj(out_dir, '%s_space-T1w_desc-brain_intersect_mask.nii.gz' % (sub))
out_file=opj(out_dir, '%s_%s_space-T1w_desc-preproc_bold_trim%dTRs_centered-masked.nii.gz' % (sub, run, n_trunc))


In [15]:
out_file

'/projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA01/sub-MONSTERA01_task-01_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz'

In [16]:
calc = Calc()

In [17]:
calc.inputs.in_file_a = trim_output
calc.inputs.in_file_b = mean_output
calc.inputs.in_file_c = mask_output

calc.inputs.expr= 'c * min(200, a/b*100)*step(a)*step(b)'
calc.inputs.out_file = out_file

In [18]:
calc.cmdline 

'3dcalc -a /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA01/sub-MONSTERA01_task-01_space-T1w_desc-preproc_bold_trim6TRs.nii.gz -b /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA01/sub-MONSTERA01_task-01_space-T1w_desc-preproc_bold_trim6TRs_mean-img.nii.gz -c /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA01/sub-MONSTERA01_space-T1w_desc-brain_intersect_mask.nii.gz -expr "c * min(200, a/b*100)*step(a)*step(b)" -prefix /projects/kuhl_lab/wanjiag/MONSTERA/derivatives/preprocess/sub-MONSTERA01/sub-MONSTERA01_task-01_space-T1w_desc-preproc_bold_trim6TRs_centered-masked.nii.gz'

In [19]:
calc.run()

OSError: No command "3dcalc" found on host n137. Please check that the corresponding package is installed.